# Python-迭代器

## 1. 为什么要使用迭代器？

解决for循环无法迭代类创建的对象的问题，在实际应用中经常要把对象转换为起来类型的数据，通过迭代器可以快速迭代出自定义的对象，并减少代码冗余

## 2. 什么是迭代？

基础中学到的list, tuple, str等类型均可使用`for ... in ...`的方法进行有序的读取，称为遍历，也称之为迭代

## 3. 对象的划分

* 不可遍历型对象，如整型、浮点型
* 可遍历型对象，如列表、元祖、字典、字符串等

## 4. 如何判断变量是否可迭代？

使用`isinstance(XX, Iterable)`判断某一变量是否可迭代

In [1]:
from collections.abc import Iterable
isinstance([1, 2, 3], Iterable)

True

### <mark><font color=red>小技巧：isinstance经常用于判断数据类型，比type更好用</font></mark>

## 5. 如何判断变量是否为迭代器类型？

In [2]:
from collections.abc import Iterator
isinstance([1, 2, 3], Iterator)

False

## 6. 什么是迭代器(Iterator)？

* 定义：可以记住遍历位置的对象
* 特点：从第一个元素开始访问，直到所有元素被访问完结束，注意只能往前而不能后退
* 迭代器的本质是记住访问过的元素的位置，所以可迭代对象的本质是其自身有迭代器对象
    * 可以通过iter()函数获取可迭代对象的迭代器（<font color=red>注意这里不是方法，函数是指iter(X)，方法则是X.iter()</font>）
    * 然后对迭代器不断使用next()函数来获得下一条数据
* for循环是此方法的封装，`for A in B`的本质是
    * 第1步：将可遍历对象B作为实参，获取其迭代器B_iter；
    * 第2步：使用next函数获取下一条数据b；
    * 第3步：将数据b存入A中；
    * 第4步：执行for循环体中的代码；
    * 第5步：一次for循环执行完毕后，重复2-4步；
        * 当B中所有数据都访问后，下一次next方法取数据会出现StopIteration异常；
        * for循环中自带异常处理，即当遇到这个异常时，自动结束for循环的运行；
        * 自己实现时可以通过`try: ... except ..... as ...`；

### <mark><font color=red>注意：iter()函数用于可迭代对象，next()函数用于迭代器！！！！</font></mark>

练习：使用iter()和next()实现for循环

In [8]:
# 1. 定义一个可迭代对象
nums = [11, 22, 33, 44] 
# 2. 获取这个可迭代对象的迭代器
nums_iter = iter(nums) # 迭代器
# 3. 循环调用next函数取迭代器中的数据
while True:
    try:
        # 4. 提取迭代器的下一个数据
        num = next(nums_iter)
        print(num)
    except StopIteration as ret:
        print("遍历完成")
        break
print("-"*30)
# 使用for循环遍历
for num in nums:
    print(num)

11
22
33
44
遍历完成
------------------------------
11
22
33
44


### <mark><font color=red>注意：不要采用根据元素总数来设计程序，如下所示，太投机取巧，最好的方式是加入try except</font></mark>

In [9]:
nums = [11, 22, 33, 44] 
nums_iter = iter(nums) 
i = 0
while i < len(nums):
    num = next(nums_iter)
    print(num)
    i += 1

11
22
33
44


# 以上均属于面向过程的编程方法，而在面向对象的编程方法中常常是在类中定义迭代方法：

## 7. 什么是可迭代对象(object)？

* 从类的角度来看，就是具有`__iter__`方法和`__next__`方法的类创建的对象称为可迭代对象
* 这是因为在使用`iter()`函数获取自定义对象的迭代器时，本质上是调用该对象所属的类中的`__iter__`方法，无论该方法具体写的是什么，都是可迭代对象

In [10]:
from collections.abc import Iterable

class MyList(object):
    def __init__(self):
        self.container = []
    
    def __iter__(self):
        pass #表示缺省
    
    def add(self, item):
        self.container.append(item)

mylist = MyList()
mylist.add(11)
mylist.add(22)
mylist.add(33)
print(isinstance(mylist, Iterable))

True


* 同理，需要`__next__`方法，也是因为使用`next()`函数时，本质上是调用迭代器对象的`__next_`_方法
* 只有`__iter__`方法的类属于可迭代对象，而`__iter__`方法和`__next__`方法都有的类属于迭代器
* 将两者关联在一起的方法就是在可迭代对象的`__iter__`方法中返回迭代器类创建的对象即可，如下所示：

In [14]:
from collections.abc import Iterable
from collections.abc import Iterator

class MyList(object):
    def __init__(self):
        self.container = []
    
    def __iter__(self):
        # 这个方法有两个功能
        # 1. 标记当前类创建出来的对象一定是可迭代对象
        # 2. 当使用系统iter()函数时，这个类内的`__iter__`方法会自动被调用，从而返回自己指定的迭代器
        return MyIterator()
    
    def add(self, item):
        self.container.append(item)
        
class MyIterator(object):
    def __init__(self):
        pass
    
    def __iter__(self):
        pass
    
    def __next__(self):
        pass
    
mylist = MyList()
mylist_iter = iter(mylist)

print("mylist是否是可迭代对象", isinstance(mylist, Iterable))
print("mylist是否是迭代器", isinstance(mylist, Iterator))
print("mylist_iter是否是可迭代对象", isinstance(mylist_iter, Iterable))
print("mylist_iter是否是迭代器", isinstance(mylist_iter, Iterator))

mylist是否是可迭代对象 True
mylist是否是迭代器 False
mylist_iter是否是可迭代对象 True
mylist_iter是否是迭代器 True


### <mark><font color=red>注意：创建可迭代对象的目的是，一方面可以使用for等遍历，另一方面可以与其他可迭代数据类型转化</font></mark>

In [16]:
class MyList(object):
    def __init__(self):
        self.container = []
    
    def __iter__(self):
        # 注意：MyList类中的__iter__方法是把当前类的self作为参数整体传给MyIterator类，
        # 所以MyIterator类的__init__方法除了自己的self以外还有一个参数，用于接受传递过来的参数，即MyList类的self
        myiterator = MyIterator(self)
        return myiterator
    
    def add(self, item):
        self.container.append(item)
        
class MyIterator(object):
    """自定义的供上面可迭代对象使用的迭代器"""
    def __init__(self, mylist):
        self.mylist = mylist
        self.current = 0
    
    def __next__(self):
        if self.current < len(self.mylist.container):
            item = self.mylist.container[self.current]
            self.current += 1
            return item
        else:
            # 注意：在写主体程序的时候通常用try execpt，而写类中的方法时通常用raise XXX 来抛出异常
            raise StopIteration
            # 注意：如果遍历完不抛出异常，而只是返回none，则无法应用于for循环
            # 因为for循环带有异常检测功能，根据__next__方法返回的异常来结束循环的，而无法判断None
            # return None
    
    def __iter__(self):
        return self
    
if __name__ == '__main__':
    mylist = MyList()
    mylist.add(1)
    mylist.add(2)
    mylist.add(3)
    mylist.add(4)
    
    # 通过以上方法，实现了一个可迭代对象，从而可以与list等可迭代对象进行类型转换
    nums = list(mylist)
    print(nums)

[1, 2, 3, 4]


### <mark><font color=red>注意：迭代器一定是可迭代对象，但可迭代对象不一定是迭代器！！！！</font></mark>

* 上述案例，可以合并迭代器和可迭代对象的类，使可迭代对象的类直接成为迭代器
* 注意：此时`__iter__`方法直接返回自己self即可，只不过要添加表示当前下标的变量，并进行初始化，图片3中只放在了`__init__`方法中, 导致只能迭代一次

In [21]:
class MyList(object):
    def __init__(self):
        self.container = []
        self.current = 0
    
    def __iter__(self):
        # 直接返回自己，因为本身就是迭代器
        return self
        
    def __next__(self):
        if self.current < len(self.container):
            item = self.container[self.current]
            self.current += 1
            return item
        else:
            raise StopIteration
    
    def add(self, item):
        self.container.append(item)
    
if __name__ == '__main__':
    mylist = MyList()
    mylist.add(1)
    mylist.add(2)
    mylist.add(3)
    mylist.add(4)
    
    for num in mylist:
        print(num)
    print("-"*30)
    # 这种写法会导致只能遍历一次！！！！
    for num in mylist:
        print(num)

1
2
3
4
------------------------------


### 小总结
* 凡是可作用于`for`循环的对象都是可迭代对象类型；
* 凡是可作用于`next()`函数的对象都是迭代器类型；
* <font color=red>集合数据类型如`list``dict`、`str`等都是可迭代对象类型，但不是迭代器类型，不过可以通过`iter()`函数获得其迭代器对象</font>
* <font color=red>除了for循环能接收可迭代对象外，列表、元祖等也可以接收可迭代对象！！！也就是通常理解上的类型转换</font>
    * 把list转为tuple，本质过程是从list中迭代取出每一个元素，然后一个一个放入元祖中，所以list和tuple之间是可以相互转换的
    * <font color=red>构造的可迭代对象也可以很容易转成列表、元祖、集合等数据类型，如下所示，即list、tuple等类型转换的核心是：被转换对象必须可迭代！！</font>

In [24]:
nums = [11, 22, 33, 44] 
print(tuple(nums))
nums = (77, 88, 99)
print(list(nums))
# 表面上看是类型转换，本质是接收可迭代对象

(11, 22, 33, 44)
[77, 88, 99]


<font color=red>改进：如果要把`__iter__`方法和`__next__`方法放在同一个类中，对于下标的初始化可以放在iter中，不过为了形式上统一，放在`__next__`方法的异常情况中，如下所示`</font>

In [25]:
class StuSystem(object):
    """
    学生管理系统
    """
    def __init__(self):
        self.stus = []
        self.current = 0

    def __iter__(self):
        # 如果self.current初始化放在这里，则__init__和__next__不用再加初始化
        # self.current = 0
        return self

    def __next__(self):
        if self.current<=len(self.stus)-1:
            item = self.stus[self.current]
            self.current+=1
            return item
        else:
            # self.current初始化放在iter中也可以，不过为了形式上的统一，放在next的异常情况中，此时必须在__init__中加入第一个初始化
            self.current = 0
            raise StopIteration


    def add(self):
        """
        添加新学生
        :return:
        """
        name = input("学生姓名:")
        id = input("学生学号:")

        new_stu = dict()
        new_stu["name"]=name
        new_stu["id"]=id

        self.stus.append(new_stu)

stu_sys = StuSystem()

stu_sys.add()
stu_sys.add()
stu_sys.add()

for stu in stu_sys:
    print(stu)

# 看一下第二次循环会不会出问题
for stu in stu_sys:
    print(stu)

stu_list = [stu for stu in stu_sys]
print(stu_list)
print(list(stu_sys))

学生姓名:liao
学生学号:1
学生姓名:yu
学生学号:2
学生姓名:xuan
学生学号:3
{'name': 'liao', 'id': '1'}
{'name': 'yu', 'id': '2'}
{'name': 'xuan', 'id': '3'}
{'name': 'liao', 'id': '1'}
{'name': 'yu', 'id': '2'}
{'name': 'xuan', 'id': '3'}
[{'name': 'liao', 'id': '1'}, {'name': 'yu', 'id': '2'}, {'name': 'xuan', 'id': '3'}]
[{'name': 'liao', 'id': '1'}, {'name': 'yu', 'id': '2'}, {'name': 'xuan', 'id': '3'}]
